In [1]:
# imports 
import emp_preprocessing as ep
import emp_fake_data as ef
import emp_metrics as em
import emp_graphics as eg
import networkx as nx
import graphics as gx
import functions as fct
import numpy as np

# data preprocessing

In [2]:
# load fake data
df_mmsr = ef.get_df_mmsr(nb_tran=5000)
df_mmsr

,report_agent_lei,cntp_lei,trns_nominal_amt,maturity_time_stamp,first_occurence,trns_type
2020-01-01 00:00:00,bank_12,bank_43,24.625484,2020-08-16 21:46:24.324844800,True,LEND
2020-01-01 05:00:00,bank_21,fund_44,63.870558,2020-03-09 19:22:32.420294400,False,LEND
2020-01-01 10:00:00,bank_19,bank_28,88.447007,2020-05-18 04:02:55.979692800,True,SELL
2020-01-01 15:00:00,bank_38,bank_46,75.800468,2020-01-23 06:23:51.785116800,False,LEND
2020-01-01 20:00:00,bank_28,bank_99,7.625742,2020-08-26 06:57:17.419536000,False,LEND
...,...,...,...,...,...,...
2022-11-06 15:00:00,bank_48,fund_26,59.412782,2023-04-23 09:08:50.018208000,False,BUYI
2022-11-06 20:00:00,bank_12,bank_75,40.562559,2023-06-23 22:18:04.840876800,False,LEND
2022-11-07 01:00:00,bank_49,bank_63,7.495949,2023-06-24 22:58:42.484022400,True,LEND
2022-11-07 06:00:00,bank_10,bank_73,70.904792,2022-12-02 16:33:55.598198400,True,SELL


In [3]:
dic_obs_adj_cr, dic_obs_adj_tr = ep.build_from_data(df_mmsr=df_mmsr)

100%|██████████| 5000/5000 [02:19<00:00, 35.92it/s] 


# metrics measurement

In [4]:
# parameters
agg_periods = [1, 50, 100, 250]
path_results = "./results/general-testing/"

## jaccard

In [5]:
dic_jaccard = em.compute_jaccard(dic_obs_adj_tr=dic_obs_adj_tr, agg_periods=agg_periods)

c:\Users\VictorLeCoz\OneDrive - Quant Decisions S.L\Documents\PhD\secured-inter-bank-network\emp_metrics.py:51: RuntimeWarning: invalid value encountered in long_scalars
  np.logical_and(


In [6]:
eg.plot_jaccard_aggregated(dic_jaccard, agg_periods, path_results)

## density

In [7]:
dic_density = em.compute_density(dic_obs_adj_tr=dic_obs_adj_tr, agg_periods=agg_periods)

In [8]:
eg.plot_network_density(dic_density, agg_periods, path_results)

## core periphery

In [10]:
bank_network = nx.from_numpy_matrix(
np.array(dic_obs_adj_tr["2020-01-05"]), parallel_edges=False, create_using=nx.DiGraph
)  # build nx object
sig_c, sig_x, significant, p_value = fct.cpnet_test(
bank_network
)  # run cpnet test
gx.plot_core_periphery(
bank_network=bank_network,
sig_c=sig_c,
sig_x=sig_x,
path=path_results,
step="2020-01-05",
name_in_title="reverse repos",
)  # plot charts

100%|██████████| 300/300 [00:26<00:00, 11.20it/s]


1 core-periphery structure(s) detected, but 0 significant, p-values are [0.37590634307784243] 
